# PoC

In [1]:
import sys

sys.path.append("..")
sys.path.append("../../inputs")

In [2]:
import logging

In [3]:
logging.basicConfig(
    # filename=__file__.replace('.py', '.log'),
    stream=sys.stdout,
    level=logging.getLevelName("INFO"),
    format="%(asctime)s [%(levelname)s] [%(module)s] %(message)s",
)

In [4]:
log = logging.getLogger(__name__)

In [5]:
from omegaconf import OmegaConf

In [6]:
c = OmegaConf.load("../config/main.yaml")

In [7]:
c.settings.debug = False
c.wandb.enabled = False
c.settings.dirs.working = ".."
c.settings.dirs.input = "../../inputs/"
c.settings.dirs.input_minimal = "../../datasets/inputs/"
c.settings.dirs.preprocess = "../../inputs/preprocess/"

c.settings.gpus = "1"

In [8]:
log.info(OmegaConf.to_yaml(c))

2022-04-01 21:31:30,840 [INFO] [3244290467] defaults:
- _self_
hydra:
  run:
    dir: ../outputs/${now:%Y-%m-%d_%H-%M-%S}
  job_logging:
    formatters:
      simple:
        format: '%(asctime)s [%(levelname)s][%(module)s] %(message)s'
wandb:
  enabled: false
  entity: imokuri
  project: ump
  dir: ${hydra:runtime.cwd}/../cache
  group: default
settings:
  print_freq: 100
  gpus: '1'
  dirs:
    working: ..
    input: ../../inputs/
    input_minimal: ../../datasets/inputs/
    feature: ${settings.dirs.input}features/
    preprocess: ../../inputs/preprocess/
  inputs:
  - train.csv
  debug: false
  n_debug_data: 100000
  amp: true
  multi_gpu: true
  training_method: nn
params:
  seed: 13440
  n_class: 1
  preprocess: []
  pca_n_components: 50
  n_fold: 5
  skip_training: false
  epoch: 10
  es_patience: 0
  batch_size: 640
  gradient_acc_step: 1
  max_grad_norm: 1000
  fold: combinational_purged
  group_name: investment_id
  time_name: time_id
  label_name: target
  use_feature: true


In [9]:
import torch

In [10]:
import src.utils as utils

In [11]:
device = utils.gpu_settings(c)

2022-04-01 21:31:32,296 [INFO] [utils] CUDA_VISIBLE_DEVICES: 1
2022-04-01 21:31:32,362 [INFO] [utils] torch device: cuda, device count: 1


## Preprocess

In [12]:
from src.load_data import InputData

2022-04-01 21:31:32,669 [INFO] [loader] Loading faiss with AVX2 support.
2022-04-01 21:31:32,694 [INFO] [loader] Successfully loaded faiss with AVX2 support.


In [13]:
input = InputData(c)

2022-04-01 21:31:32,805 [INFO] [load_data] Load feather file. path: ../../inputs/train.f
2022-04-01 21:36:44,808 [INFO] [utils] Mem. usage decreased to 366Mb: 49% reduction


In [14]:
input.train.describe()

,time_id,investment_id,target,f_0,f_1,f_2,f_3,f_4,f_5,f_6,...,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299,group_fold,time_fold
count,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06,...,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06,3.141410e+06
mean,6.654862e+02,1.885265e+03,-2.109159e-02,1.637057e-03,-1.257678e-02,8.514749e-03,-2.003703e-02,-5.416438e-03,-3.034008e-03,1.934330e-03,...,-6.509154e-03,2.851608e-03,-4.730820e-03,-2.773806e-02,-1.517383e-03,-7.057928e-03,-4.216896e-03,-4.319488e-03,1.999997e+00,2.499998e+00
std,3.560434e+02,1.083616e+03,9.176629e-01,1.079721e+00,1.030980e+00,1.030281e+00,9.602916e-01,9.895188e-01,1.104578e+00,1.067755e+00,...,1.070176e+00,1.101158e+00,1.144516e+00,9.592550e-01,1.140459e+00,1.108097e+00,1.051252e+00,1.008220e+00,1.414214e+00,1.722291e+00
min,0.000000e+00,0.000000e+00,-9.419646e+00,-1.765789e+01,-6.579473e+00,-8.644268e+00,-1.800427e+01,-4.000015e+00,-8.833704e+00,-6.384251e+00,...,-8.183732e+00,-2.399478e+01,-1.497270e+01,-9.348986e+00,-7.366648e+00,-7.579406e+00,-6.707284e+00,-1.028264e+01,0.000000e+00,0.000000e+00
25%,3.530000e+02,9.520000e+02,-5.004572e-01,-4.071441e-01,-6.813878e-01,-6.530838e-01,-4.496046e-01,-3.532303e-01,-7.070855e-01,-7.911198e-01,...,-6.981400e-01,-1.377131e-01,-9.397985e-01,-5.147926e-01,-9.573807e-01,-7.239380e-01,-8.165390e-01,-5.080143e-01,1.000000e+00,1.000000e+00
50%,7.040000e+02,1.882000e+03,-9.717009e-02,2.431158e-01,-4.867587e-02,4.350941e-02,-2.639937e-01,-1.908876e-01,-2.144796e-02,1.437945e-02,...,-1.910102e-01,2.164071e-01,2.087202e-01,-2.907780e-01,6.955573e-03,-1.650222e-01,2.300689e-02,-2.824031e-01,2.000000e+00,2.000000e+00
75%,9.750000e+02,2.830000e+03,3.572908e-01,6.649507e-01,6.086557e-01,6.587324e-01,7.763371e-02,2.784694e-02,6.835684e-01,8.008306e-01,...,4.841421e-01,5.088849e-01,8.616266e-01,1.275946e-01,9.503851e-01,6.098197e-01,7.984827e-01,1.411301e-01,3.000000e+00,4.000000e+00
max,1.219000e+03,3.773000e+03,1.203861e+01,7.845261e+00,8.707207e+00,8.009340e+00,4.706333e+01,7.662866e+01,7.646200e+00,6.778142e+00,...,2.231185e+01,6.587691e+00,6.978151e+00,6.140367e+01,7.679950e+00,1.241804e+01,7.003982e+00,4.337021e+01,4.000000e+00,5.000000e+00


## Fold

In [15]:
input.train.fillna({"time_fold": c.params.n_fold}, inplace=True)

In [16]:
input.train

,row_id,time_id,investment_id,target,f_0,f_1,f_2,f_3,f_4,f_5,...,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299,group_fold,time_fold
0,0_1,0,1,-0.300875,0.932573,0.113691,-0.402206,0.378386,-0.203938,-0.413469,...,0.200075,0.819155,0.941183,-0.086764,-1.087009,-1.044826,-0.287605,0.321566,1.0,0.0
1,0_2,0,2,-0.231040,0.810802,-0.514115,0.742368,-0.616673,-0.194255,1.771210,...,-0.734579,0.819155,0.941183,-0.387617,-1.087009,-0.929529,-0.974060,-0.343624,3.0,0.0
2,0_6,0,6,0.568807,0.393974,0.615937,0.567806,-0.607963,0.068883,-1.083155,...,-0.551904,-1.220772,-1.060166,-0.219097,-1.087009,-0.612428,-0.113944,0.243608,0.0,0.0
3,0_7,0,7,-1.064780,-2.343535,-0.011870,1.874606,-0.606346,-0.586827,-0.815737,...,-0.266359,-1.220772,0.941183,-0.609113,0.104928,-0.783423,1.151730,-0.773309,4.0,0.0
4,0_8,0,8,-0.531940,0.842057,-0.262993,2.330030,-0.583422,-0.618392,-0.742814,...,-0.741355,-1.220772,0.941183,-0.588445,0.104928,0.753279,1.345611,-0.737624,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3141405,1219_3768,1219,3768,0.033600,0.093530,-0.720275,-0.345497,-0.438781,-0.166972,-0.437182,...,-0.660579,0.875537,0.421628,-0.428097,-0.075548,-0.533092,-0.193732,-0.581394,2.0,5.0
3141406,1219_3769,1219,3769,-0.223264,-1.344935,-0.199987,-0.107702,-0.454677,-0.221914,-0.141174,...,-0.670493,0.875537,0.421628,-0.729949,-1.514277,0.013145,-0.890270,-0.589705,2.0,5.0
3141407,1219_3770,1219,3770,-0.559415,0.979489,-1.110491,1.006980,-0.467307,-0.159549,1.355671,...,0.820784,-1.142157,0.421628,-0.363329,1.363181,-0.079106,-1.580124,-0.297625,2.0,5.0
3141408,1219_3772,1219,3772,0.009599,-2.565332,0.320301,0.076600,1.380182,-0.155366,-0.689000,...,0.133074,-1.142157,0.421628,-0.375288,-1.514277,-0.973762,0.608647,-0.372040,0.0,5.0


In [17]:
input.train["group_fold"].value_counts()

1.0    628285
0.0    628283
4.0    628281
3.0    628281
2.0    628280
Name: group_fold, dtype: int64

In [18]:
input.train["time_fold"].value_counts()

0.0    523570
1.0    523568
5.0    523568
4.0    523568
3.0    523568
2.0    523568
Name: time_fold, dtype: int64

In [19]:
input.train[["group_fold", "time_fold"]].value_counts()

group_fold  time_fold
1.0         0.0          107189
0.0         3.0          106248
1.0         2.0          105776
3.0         1.0          105700
2.0         1.0          105484
4.0         1.0          105467
3.0         3.0          105462
            5.0          105437
            4.0          105423
0.0         4.0          105269
2.0         0.0          105064
4.0         5.0          104911
            0.0          104903
0.0         5.0          104838
            2.0          104803
4.0         2.0          104687
2.0         4.0          104554
            2.0          104495
            5.0          104380
1.0         4.0          104375
4.0         3.0          104366
2.0         3.0          104303
1.0         5.0          104002
0.0         0.0          103962
4.0         4.0          103947
3.0         2.0          103807
1.0         1.0          103754
            3.0          103189
0.0         1.0          103163
3.0         0.0          102452
dtype: int64

In [20]:
from src.make_fold import train_test_split

In [21]:
train_folds, valid_folds = train_test_split(c, input.train, 1)

2022-04-01 21:37:12,849 [INFO] [make_fold] Num of training data: 2091784, num of validation data: 1047138


In [22]:
len(train_folds)

2091784

In [23]:
len(valid_folds)

1047138

## Model

In [24]:
from src.make_model import make_model

In [25]:
c.params.model = "ump_ad_ae"

In [26]:
model = make_model(c, device)

In [27]:
model

DataParallel(
  (module): AnomalyDetectionAutoEncoderModel(
    (encoder): Sequential(
      (0): Linear(in_features=300, out_features=200, bias=True)
      (1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=200, out_features=100, bias=True)
    )
    (decoder): Sequential(
      (0): Linear(in_features=100, out_features=200, bias=True)
      (1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=200, out_features=300, bias=True)
    )
  )
)

In [28]:
input_ = torch.randn(c.params.batch_size, c.params.model_input).to(device)
# input_ = torch.randn(c.params.batch_size, c.params.model_input * 10).to(device)
# input_ = torch.randn(c.params.batch_size, c.params.model_window, c.params.model_input).to(device)

# num_gpu = len(c.settings.gpus.split(","))
# hidden = torch.zeros(1, c.params.batch_size // num_gpu, 300).to(device)
# cell = torch.zeros(1, c.params.batch_size // num_gpu, 300).to(device)
# hidden_cell = (hidden, cell)

In [29]:
input_.size()

torch.Size([640, 300])

In [32]:
# res = model(input_)
# res, hidden_cell = model(input_, hidden_cell)
res, hidden_cell = model(input_)

In [35]:
# res.shape
res.shape, hidden_cell.shape

(torch.Size([640, 300]), torch.Size([640, 100]))

In [34]:
res

tensor([[-0.5015,  0.3572, -0.0763,  ..., -0.0549,  0.0104, -0.1072],
        [-0.6294,  0.3574, -0.1827,  ...,  0.3806,  0.6675,  0.6406],
        [ 0.5029,  0.6001,  0.5078,  ..., -0.3198, -0.0655, -0.0076],
        ...,
        [ 0.5005,  0.6494,  0.4521,  ..., -0.4268,  0.0780,  0.3391],
        [-0.5020,  0.2466, -0.4644,  ..., -0.1655, -0.2642, -0.0811],
        [-0.0433, -0.3599,  0.3828,  ..., -0.3982, -0.2294,  0.2401]],
       device='cuda:0', dtype=torch.float16, grad_fn=<AddmmBackward>)

In [36]:
# hidden_cell

tensor([[ 0.0370, -0.0634,  0.2133,  ...,  0.1368,  0.3916,  0.1605],
        [ 0.1716, -0.4827, -0.5283,  ...,  0.1445,  0.0398, -0.2106],
        [ 0.2363, -0.7349, -0.1589,  ...,  0.1139,  0.2583,  0.1687],
        ...,
        [ 0.3188, -0.8257,  0.0388,  ...,  0.1697, -0.2642, -0.2209],
        [ 1.3926,  0.2871, -0.1774,  ...,  0.2666,  0.9268, -0.5371],
        [ 0.1702, -0.1589, -0.4749,  ..., -0.1714, -0.5991,  0.3679]],
       device='cuda:0', dtype=torch.float16, grad_fn=<AddmmBackward>)